In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
import json
import csv

load_dotenv(".env")

In [ ]:
chat = ChatGroq(temperature=0, groq_api_key=os.getenv("API_Key"), model_name="mixtral-8x7b-32768")

In [ ]:
system_prompt = "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn.\n"
system_prompt += "Câu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực."
system_prompt += "Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác. Nếu bạn không biết câu trả lời cho một câu hỏi, hãy trẳ lời là bạn không biết và vui lòng không chia sẻ thông tin sai lệch."
human = """Trả về thông tin được trích xuất dưới định dạng JSON sau:
{{
    "Giá cả": "null",
    "Diện tích": "null",
    "Hướng nhà": "null",
    "Hướng ban công": "null",
    "Số phòng ngủ": "null",
    "Số nhà vệ sinh": "null",
    "Vị trí": "null",
    "Số tầng": "null",
    "Giấy tờ rõ ràng": 0,
    "Hỗ trợ vay vốn ngân hàng": 0,
    "Bàn giao nội thất": 0
}}

Điền các thông tin trích xuất được từ {question} theo như mô tả trên. Chỉ trả về thông tin như mô tả trên, không cần giải thích. Lưu ý chắc chắn tên của các keys phải giống chính xác như mô tả trên.\
Số tiền xuất ra dưới đơn vị hàng tỷ (ví dụ: thu được 24 tỷ trả về 24 tỷ, thu được 24000000000 vẫn trả về 24 tỷ)\
Thông tin về vị trí nếu là tên chung cư hãy tìm kiếm và điền vào cả tên chung cư và địa chỉ của chung cư đó. \
Thông tin về số phòng ngủ có thể ghi tắt là "PN" và nhà vệ sinh là "WC" \
Thông tin về giấy tờ rõ ràng điền 1 nếu phát hiện cụm từ "sổ hồng", "sổ đỏ", "sổ vuông", điền 0 nếu không có. Thông tin về hỗ trợ vay vốn ngân hàng điền 1 nếu phát hiện cụm từ "vay vốn", "trả góp", "ngân hàng", điền 0 nếu không có. \
Thông tin về bàn giao nội thất điền 1 nếu phát hiện cụm từ "nội thất" hoặc tên các loại nội thất, điền 0 nếu không có. Dùng "null" để thay thế những thông tin không có, và 0 hoặc 1 để đánh dấu các trường có hoặc không có."""


In [ ]:
def transform_json(input_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    result = {
        "Link": {},
        "Price": {},
        "Parameters": {},
        "Details": {},
    }
    
    for i, entry in enumerate(data):
        result["Link"][i] = entry["Link"]
        result["Price"][i] = entry["Price"]
        result["Parameters"][i] = entry["Parameters"]
        result["Details"][i] = entry["Details"]
    return result

def format_observations(transformed_data):
    formatted_data = []
    for i in transformed_data["Link"].keys():
        observation = {
            "Link": transformed_data["Link"][i],
            "Price": transformed_data["Price"][i],
            "Parameters": transformed_data["Parameters"][i],
            "Details": transformed_data["Details"][i],
        }
        formatted_data.append(observation)
    return formatted_data

def format_none_as_null(data):
    for key, value in data.items():
        if value is None:
            data[key] = "null"
    return data

def write_to_csv(final_data, output_file):
    # Define CSV column names
    fieldnames = ["Giá cả", "Diện tích", "Hướng nhà", "Hướng ban công", "Số phòng ngủ", "Số nhà vệ sinh", "Vị trí",
                   "Số tầng", "Giấy tờ rõ ràng", "Hỗ trợ vay vốn ngân hàng", "Bàn giao nội thất"]
    
    # Parse the JSON-like string into a dictionary
    data_dict = json.loads(final_data.replace("'", "\""))
    format_none_as_null(data_dict)

    # Write to CSV file
    with open(output_file, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write the header if the file is empty
        if csvfile.tell() == 0:
            writer.writeheader()
        
        # Write the data
        writer.writerow(data_dict)

# Example usage
input_file = 'small_file_23.json'
output_file = 'chung_cu_23.csv'
transformed_data = transform_json(input_file)
formatted_data = format_observations(transformed_data)
prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", human)])
chain = prompt | chat

for data in formatted_data:
    question = """
    {{
        "Link": "{Link}",
        "Price": "{Price}",
        "Parameters": "{Parameters}",
        "Details": "{Details}"
    }}
    """.format(
        Link=data["Link"],
        Price=data["Price"],
        Parameters=data["Parameters"],
        Details=data["Details"]
    )
    final_data = chain.invoke({"question": question})
    try:
        write_to_csv(final_data.content, output_file)
    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {e}")
        print(f"Invalid JSON: {final_data.content}")
    
print(f"Data has been written to {output_file}")

In [8]:
import pandas as pd

# Đường dẫn tới các file CSV
files = [
    'chung_cu_17.csv',
    'chung_cu_18.csv',
    'chung_cu_19.csv',
    'chung_cu_20.csv',
    'chung_cu_21.csv',
    'chung_cu_22.csv',
    'chung_cu_23.csv'
]

# Đọc và gộp các file CSV
dataframes = [pd.read_csv(file, encoding='utf-8') for file in files]
combined_df = pd.concat(dataframes, ignore_index=True)

# Ghi dữ liệu vào file Excel
output_file = 'chung_cu_17_23.xlsx'
combined_df.to_excel(output_file, index=False)
